In [25]:
import joblib
import pandas as pd
import numpy as np
import json
import re
from transformers import BertTokenizer, BertModel
import nltk
import torch
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/cpan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
best_xgb = joblib.load('/Users/cpan/Desktop/project/notebooks/tuned_xgb_model1.pkl')

In [5]:
json_file_path = '/Users/cpan/OneDrive - Guardant Health/clean_test_results/labeled_test_results.json'
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

In [8]:
def create_dataframe(data):
    dataframe = []
    for entry in data:
        file_name = entry['file_name']
        patient_id = entry['patient_id']
        page_content = entry['page_content']
        tests = entry['tests']    
        dataframe.append({'file_name': file_name, 'patient_id': patient_id, 'page_content': page_content})
    df = pd.DataFrame(dataframe)
    return df

In [9]:
df_data = create_dataframe(data)

In [10]:
print(df_data.shape)

(2793, 3)


In [19]:
# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters, digits, and extra spaces
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Define your custom vocabulary to be added to stopwords
    custom_vocab = ["redacted", "redactedredacted", 'is', 'to', 'of', 'with', 'in', 'no']  # Add your domain-specific terms
    
    # Combine NLTK stopwords and custom vocabulary
    stop_words = set(stopwords.words('english') + custom_vocab)
    
    words = text.split()
    # filter out words that have length less than 2 
    filtered_words = [word for word in words if len(word) > 2 and word not in stop_words]
    text = ' '.join(filtered_words)
    
    return text
# Function to extract BERT embeddings for text data
def extract_bert_features(df):
    embeddings = []
    for text in df['page_content']:
        cleaned_text = clean_text(text)
        inputs = tokenizer(cleaned_text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_pooling = torch.mean(last_hidden_states, dim=1)
        embeddings.append(avg_pooling.squeeze().numpy())
    return np.vstack(embeddings)

In [ ]:
test_features = extract_bert_features(df_data)
X_test = test_features
scaler = StandardScaler()
X_test_scaled = scaler.transform(X_test)
y_pred_tuned = best_xgb_tuned.predict(X_test_scaled)

In [ ]:
df_data['is_test'] = y_pred_tuned 

In [ ]:
# export prediction dataframe to a csv file 
save_dir = '/Users/cpan/OneDrive - Guardant Health/page_classification_data/'
train_df.to_csv(save_dir+"predictions_first3000_docs.csv", index=False)